In [1]:
import Cython
%load_ext cython

In [50]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp -a
import numpy as np
cimport numpy as np

cdef class LinearModel:
    cdef np.ndarray y
    cdef np.ndarray X
    cdef np.ndarray w
    cdef double alpha
    cdef np.ndarray prior
    cdef size_t n
    cdef size_t m
    cdef double log2pi
    
    def __init__(self, y, X, alpha = 0.01):
        self.y = y
        self.X = X
        self.alpha = alpha
        self.w = None

        self.prior = np.random.multivariate_normal(mean = np.zeros(X.shape[1]), cov = alpha*np.eye(X.shape[1]))
        
        self.n = X.shape[1]
        self.m = y.shape[0]
        self.log2pi = np.log(2*np.pi)

    cpdef double[:] fit(self) nogil:
        cdef double[:, :] prior = self.alpha*np.eye(self.n, dtype=np.float)
        cdef double[:, :] x_t_x = self.X.T@self.X
        cdef double[:, :] under_inv = prior + x_t_x
        cdef double[:, :] inv = np.linalg.inv(under_inv)
        cdef double[:, :] x_y = self.X.T@self.y
        self.w = inv@x_y
        return self.w

    cpdef double[:] predict(self, double[:] params = None, double[:, :] X = None):
        if params is None:
            params=self.w
        if X is None:
            X = self.X
        return X@params



Error compiling Cython file:
------------------------------------------------------------
...
        self.n = X.shape[1]
        self.m = y.shape[0]
        self.log2pi = np.log(2*np.pi)

    cpdef double[:] fit(self) nogil:
        cdef np.ndarray[np.float, ndim=2] prior = self.alpha*np.eye(self.n, dtype=np.float)
            ^
------------------------------------------------------------

/home/local/AP-TEAM/grabovoy/.cache/ipython/cython/_cython_magic_cf4677e07ff778f9696f00a9c58baca1.pyx:27:13: Assignment of Python object not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
        self.m = y.shape[0]
        self.log2pi = np.log(2*np.pi)

    cpdef double[:] fit(self) nogil:
        cdef np.ndarray[np.float, ndim=2] prior = self.alpha*np.eye(self.n, dtype=np.float)
        cdef np.ndarray[np.float, ndim=2] x_t_x = self.X.T@self.X
            ^
------------------------------------------------------------

/home/local

In [40]:
import numpy as np

In [41]:
X = np.random.randn(15, 2)
w = np.random.randn(2)
y = X@w + 0.01*np.random.randn(15)

In [42]:
w

array([-0.75121619,  0.21177896])

In [43]:
model = LinearModel(y, X)

In [44]:
model.fit()

<MemoryView of 'ndarray' at 0x7fd4bb9b9540>

In [32]:
model.predict()

TypeError: unsupported operand type(s) for @: '_cython_magic_95dab6f478c97c1127112e282bc1c2d6._memoryviewslice' and 'NoneType'